<a href="https://colab.research.google.com/github/sundarbsb/machine-Learning/blob/master/CustomerSegmentation_and_RFM_Analysis_on_E_commerce_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**`Importing modules`**


In [ ]:
# Importation of useful libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

import random 
import datetime as dt
import re
import pickle
import nltk, warnings
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from string import digits, punctuation



from scipy.stats import chi2_contingency

from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from wordcloud import WordCloud, STOPWORDS

warnings.filterwarnings("ignore")
plt.style.use('bmh')
%matplotlib inline
import os
print(os.listdir())

**`Data Cleaning`**


In [ ]:
data = pd.read_csv("transaction_data.csv", engine="python", encoding="ISO-8859-1", error_bad_lines=False)
data.head(10)
# data.info()

Add the column - sales

In [ ]:
data['sales'] = data['NumberOfItemsPurchased'] * data['CostPerItem']

Graph of missing values

In [ ]:
plt.figure(figsize=(5, 5))
data.isnull().mean(axis=0).plot.barh()
plt.title("Ratio of missing values per columns")

It appears that we can't replace the missing values and we can't keep data without the value in the descriptions columns. So I'll simply delete the rows which have NaN values

In [ ]:
data = data.dropna(subset=["ItemDescription"])

So once again plot the missing values graph

In [ ]:
plt.figure(figsize=(5, 5))
data.isnull().mean(axis=0).plot.barh()
plt.title("Ratio of missing values per columns")

There are no more missing values. I'll now check the dupplicate values and drop them if there's any.

In [ ]:
print('Dupplicate entries: {}'.format(data.duplicated().sum()))
data.drop_duplicates(inplace = True)

In [ ]:
data=data[data.UserId!=-1]

Remove Quantity with negative values as it might be either mistakes or cancelled order

In [ ]:
data=data[data.NumberOfItemsPurchased>0]

In [ ]:
data=data[data.ItemCode > 0]

In [ ]:
data[data['CostPerItem'] < 0]

So there is no negative value for cost per item

**Transaction Time** Changing the format

In [ ]:
data['TransactionTime']=pd.to_datetime(data['TransactionTime'],errors='ignore')

In [ ]:
data.TransactionTime.describe()

**`Exploratory Data Analysis`**

**Country**

In [ ]:
data.Country.nunique()

In [ ]:
# customer_country=data[['Country','UserId']].drop_duplicates()
# customer_country.groupby(['Country'])['UserId'].aggregate('count').reset_index().sort_values('UserId', ascending=False)

uk_count = data[data['Country'] == 'United Kingdom']['Country'].count()
all_count = data['Country'].count()
uk_perc = uk_count/all_count
print(str('{0:.2f}%').format(uk_perc*100))

Therefore, nearly 89% of the sales are from United Kingdom

**Checking out columns separately**

In [ ]:
item_counts = data['ItemDescription'].value_counts().sort_values(ascending=False).iloc[0:15]
plt.figure(figsize=(18,6))
sns.barplot(item_counts.index, item_counts.values, palette=sns.cubehelix_palette(15))
plt.ylabel("Counts")
plt.title("Which items were bought more often?");
plt.xticks(rotation=90);

Best Selling product is **white hanging heart t-light holder**

Now lets analyse description

Quick check of the case sensitive of letters in Description whether it says that there are some units with lower case letters in their name or not and also check that lower case records are for canceled items or not. Here we can understand that data management in the store can be improved.

In [ ]:
data[~data['ItemDescription'].str.isupper()]['ItemDescription'].value_counts().head()

In [ ]:
lcase_counts = data[~data['ItemDescription'].str.isupper()]['ItemDescription'].value_counts().sort_values(ascending=False).iloc[0:15]
plt.figure(figsize=(18,6))
sns.barplot(lcase_counts.index, lcase_counts.values, palette=sns.color_palette("hls", 15))
plt.ylabel("Counts")
plt.title("Not full upper case items");
plt.xticks(rotation=90);

Also checking for Transactions Id's

In [ ]:
data['TransactionId'].value_counts().tail()

So, Therefore it is confirmed that every Transaction Id and description wont show any kind of information on cancelled items

**Now, Lets check for Outliers**

In [ ]:
data.describe()

The minimum value for the unitprice is 0, let's see why is that.

In [ ]:
data[(data['CostPerItem'] == 0)].head(5)



I am tempted to replace the null values by the most common one but it might be a special discount or something else so I'll leave it like that

**Now Let's do Feature Engineering**

In [ ]:
def unique_counts(data):
   for i in data.columns:
       count = data[i].nunique()
       print(i, ": ", count)
unique_counts(data)

Now Lets see sales(Total Revenue) of each country

In [ ]:
revenue_per_countries = data.groupby(["Country"])["sales"].sum().sort_values()
revenue_per_countries.plot(kind='barh', figsize=(15,12))
plt.title("Revenue per Country") 

Lets see total No of item codes of each country

In [ ]:
No_invoice_per_country = data.groupby(["Country"])["ItemCode"].count().sort_values()
No_invoice_per_country.plot(kind='barh', figsize=(15,12))
plt.title("Number of Invoices per Country")



This is very interesting since we can see that Netherlands is the 2nd country in value even though it has less invoices than countries like Germany or France for example and nearly 10 times less customers

Now Lets Label countries

In [ ]:
le = LabelEncoder()
le.fit(data['Country'])
l = [i for i in range(37)]
dict(zip(list(le.classes_),l))

In [ ]:
data['Country'] = le.transform(data['Country'])

In [ ]:
with open('labelencoder.pickle', 'wb') as g:
    pickle.dump(le, g)

In [ ]:
data.head(5)

RFM Principle

In [ ]:
data['TransactionTime'].min()

In [ ]:
data['TransactionTime'].max()

Here Outlier is there in Transaction Time, I'll change it now

In [ ]:
data=data[data.TransactionTime<"2019-03-01 13:19:00"]
data['TransactionTime'].max()

In [ ]:
data['TransactionTime'] = pd.to_datetime(data['TransactionTime'])
NOW = dt.datetime(2019, 2, 21)

In [ ]:
custom_aggregation = {}
custom_aggregation["TransactionTime"] = lambda x:x.iloc[0]
custom_aggregation["UserId"] = lambda x:x.iloc[0]
custom_aggregation["sales"] = "sum"


rfmTable = data.groupby("TransactionId").agg(custom_aggregation)

In [ ]:
rfmTable["Recency"] = NOW - rfmTable["TransactionTime"]
rfmTable["Recency"] = pd.to_timedelta(rfmTable["Recency"]).astype("timedelta64[D]")

In [ ]:
rfmTable.head()

In [ ]:
custom_aggregation = {}

custom_aggregation["Recency"] = ["min", "max"]
custom_aggregation["TransactionTime"] = lambda x: len(x)
custom_aggregation["sales"] = "sum"

rfmTable_final = rfmTable.groupby("UserId").agg(custom_aggregation)

In [ ]:
rfmTable_final.columns = ["min_recency", "max_recency", "frequency", "monetary_value"]

In [ ]:
rfmTable_final.frequency.describe()

In [ ]:
# rfmTable_final.head(5)
x_customer = rfmTable_final[rfmTable_final['frequency']==205]
x_customer.head()

In [ ]:
x_customer = data[data['UserId']==267708]
x_customer.head(5)

Therefore (267708) brought a lot of products(205)





In [ ]:
quantiles = rfmTable_final.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [ ]:
segmented_rfm = rfmTable_final

In [ ]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

Here we'll apply a score on each feature of RFM


In [ ]:
segmented_rfm['r_quartile'] = segmented_rfm['min_recency'].apply(RScore, args=('min_recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
# segmented_rfm.head()

Finally we'll set a score for each customer in the database.

In [ ]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False)

Here we have an example of customers with a score of 111 which means that they are classified as our best customers.

In [ ]:
segmented_rfm = segmented_rfm.reset_index()
segmented_rfm.head(5)

In [ ]:
data = pd.merge(data,segmented_rfm, on='UserId')

In [ ]:
data.columns

As we calculated our final RMS score now we can drop individual quartiles to make our data simple

In [ ]:
data = data.drop(columns=['r_quartile', 'f_quartile', 'm_quartile'])

Our RMS analysis is done

Clear Analysis Report will be given in Explanation.



Now Lets do K-means clustering

Creating Time Features and checking if there are any paterns due to seasonality.

In [ ]:
data['Month'] = data["TransactionTime"].map(lambda x: x.month)

In [ ]:
data['Month'].value_counts()

In [ ]:
data['Weekday'] = data["TransactionTime"].map(lambda x: x.weekday())
data['Day'] = data["TransactionTime"].map(lambda x: x.day)
data['Hour'] = data["TransactionTime"].map(lambda x: x.hour)

Product Categories

In [ ]:
X = data["ItemDescription"].unique()

stemmer = nltk.stem.porter.PorterStemmer()
stopword = nltk.corpus.stopwords.words('english')

In [ ]:
def stem_and_filter(doc):
    tokens = [stemmer.stem(w) for w in analyzer(doc)]
    return [token for token in tokens if token.isalpha()]

analyzer = TfidfVectorizer().build_analyzer()
CV = TfidfVectorizer(lowercase=True, stop_words="english", analyzer=stem_and_filter, min_df=0.00, max_df=0.3)  # we remove words if it appears in more than 30 % of the corpus (not found stopwords like Box, Christmas and so on)
TF_IDF_matrix = CV.fit_transform(X)
print("TF_IDF_matrix :", TF_IDF_matrix.shape, "of", TF_IDF_matrix.dtype)

In [ ]:
svd = TruncatedSVD(n_components = 100)
normalizer = Normalizer(copy=False)

TF_IDF_embedded = svd.fit_transform(TF_IDF_matrix)
TF_IDF_embedded = normalizer.fit_transform(TF_IDF_embedded)
print("TF_IDF_embedded :", TF_IDF_embedded.shape, "of", TF_IDF_embedded.dtype)

In [ ]:
score_tfidf = []

x = list(range(5, 155, 10))

for n_clusters in x:
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=10)
    kmeans.fit(TF_IDF_embedded)
    clusters = kmeans.predict(TF_IDF_embedded)
    silhouette_avg = silhouette_score(TF_IDF_embedded, clusters)

    rep = np.histogram(clusters, bins = n_clusters-1)[0]
    score_tfidf.append(silhouette_avg)

In [ ]:
plt.figure(figsize=(20,16))

plt.subplot(2, 1, 1)
plt.plot(x, score_tfidf, label="TF-IDF matrix")
plt.title("Evolution of the Silhouette Score")
plt.legend()

The highest value for the silhouette score is when there are 105 clusters. So we'll chose this value.

In [ ]:
n_clusters = 105

kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30, random_state=0)
proj = kmeans.fit_transform(TF_IDF_embedded)
clusters = kmeans.predict(TF_IDF_embedded)
plt.figure(figsize=(10,10))
plt.scatter(proj[:,0], proj[:,1], c=clusters)
plt.title("ACP with 105 clusters", fontsize="20")

In [ ]:
tsne = TSNE(n_components=2)
proj = tsne.fit_transform(TF_IDF_embedded)

plt.figure(figsize=(10,10))
plt.scatter(proj[:,0], proj[:,1], c=clusters)
plt.title("Visualization of the clustering with TSNE", fontsize="20")

In [ ]:
plt.figure(figsize=(20,8))
wc = WordCloud()

for num, cluster in enumerate(random.sample(range(100), 12)) :
    plt.subplot(3, 4, num+1)
    wc.generate(" ".join(X[np.where(clusters==cluster)]))
    plt.imshow(wc, interpolation='bilinear')
    plt.title("Cluster {}".format(cluster))
    plt.axis("off")
plt.figure()

In [ ]:
pd.Series(clusters).hist(bins=100)

In [ ]:
dict_article_to_cluster = {article : cluster for article, cluster in zip(X, clusters)}

In [ ]:
with open('product_clusters.pickle', 'wb') as h:
    pickle.dump(dict_article_to_cluster, h)

**Creating customer categories**




Intermediate dataset grouped by invoices

In [ ]:
cluster = data['ItemDescription'].apply(lambda x : dict_article_to_cluster[x])
df2 = pd.get_dummies(cluster, prefix="Cluster").mul(data["sales"], 0)
df2 = pd.concat([data['TransactionId'], df2], axis=1)
df2_grouped = df2.groupby('TransactionId').sum()

In [ ]:
custom_aggregation = {}
custom_aggregation["sales"] = lambda x:x.iloc[0]
custom_aggregation["min_recency"] = lambda x:x.iloc[0]
custom_aggregation["max_recency"] = lambda x:x.iloc[0]
custom_aggregation["frequency"] = lambda x:x.iloc[0]
custom_aggregation["monetary_value"] = lambda x:x.iloc[0]
custom_aggregation["UserId"] = lambda x:x.iloc[0]
custom_aggregation["NumberOfItemsPurchased"] = "sum"
custom_aggregation["Country"] = lambda x:x.iloc[0]


df_grouped = data.groupby("TransactionId").agg(custom_aggregation)

 Final dataset grouped by customers


In [ ]:
df2_grouped_final = pd.concat([df_grouped['UserId'], df2_grouped], axis=1).set_index("UserId").groupby("UserId").sum()
df2_grouped_final = df2_grouped_final.div(df2_grouped_final.sum(axis=1), axis=0)
df2_grouped_final = df2_grouped_final.fillna(0)

In [ ]:
data.columns

In [ ]:
custom_aggregation = {}
custom_aggregation["sales"] = ['min','max','mean']
custom_aggregation["min_recency"] = lambda x:x.iloc[0]
custom_aggregation["max_recency"] = lambda x:x.iloc[0]
custom_aggregation["frequency"] = lambda x:x.iloc[0]
custom_aggregation["monetary_value"] = lambda x:x.iloc[0]
custom_aggregation["NumberOfItemsPurchased"] = "sum"
custom_aggregation["Country"] = lambda x:x.iloc[0]

df_grouped_final = df_grouped.groupby("UserId").agg(custom_aggregation)

In [ ]:
df_grouped_final.columns = ["min", "max", "mean", "min_recency", "max_recency", "frequency", "monetary_value", "NumberOfItemsPurchased", "Country"]

In [ ]:
df_grouped_final.head(5)

In [ ]:
df2_grouped_final.head(5)

Now lets Cluster Customers

In [ ]:
X1 = df_grouped_final.to_numpy()
X2 = df2_grouped_final.to_numpy()

scaler = StandardScaler()
X1 = scaler.fit_transform(X1)
X_final_std_scale = np.concatenate((X1, X2), axis=1)

In [ ]:
x = list(range(2, 12))
y_std = []
for n_clusters in x:
    print("n_clusters =", n_clusters)
    
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=10)
    kmeans.fit(X_final_std_scale)
    clusters = kmeans.predict(X_final_std_scale)
    silhouette_avg = silhouette_score(X_final_std_scale, clusters)
    y_std.append(silhouette_avg)
    print("The average silhouette_score is :", silhouette_avg, "with Std Scaling")

We want to have at least 5, 6 clusters so we won't take 2 or 3 clusters even though they have the highest silhouette scores, 8 clusters would fit the best here.

In [ ]:
kmeans = KMeans(init='k-means++', n_clusters = 8, n_init=30, random_state=0)
kmeans.fit(X_final_std_scale)
clusters = kmeans.predict(X_final_std_scale)

In [ ]:
plt.figure(figsize = (20,8))
n, bins, patches = plt.hist(clusters, bins=8)
plt.xlabel("Cluster")
plt.ylabel("No.of Customers")
plt.title("Number of customers per cluster")
plt.xticks([rect.get_x()+ rect.get_width() / 2 for rect in patches], ["Cluster {}".format(x) for x in range(8)])

for rect in patches:
    y_value = rect.get_height()
    x_value = rect.get_x() + rect.get_width() / 2

    space = 5
    va = 'bottom'
    label = str(int(y_value))
    
    plt.annotate(
        label,                      
        (x_value, y_value),         
        xytext=(0, space),          
        textcoords="offset points", 
        ha='center',                
        va=va)

In [ ]:
df_grouped_final["cluster"] = clusters

In [ ]:
final_dataset = pd.concat([df_grouped_final, df2_grouped_final], axis = 1)
final_dataset.head()

In [ ]:
final_dataset_V2 = final_dataset.reset_index()

In [ ]:
# final_dataset_V2.to_csv("final_dataset_V2.csv",index=False)

In [ ]:
with open('df_cleaned.pickle', 'wb') as f:
    pickle.dump(data, f)

Now Lets Interpret Clusters

In [ ]:
tsne = TSNE(n_components=2)
proj = tsne.fit_transform(X_final_std_scale)

plt.figure(figsize=(10,10))
plt.scatter(proj[:,0], proj[:,1], c=clusters)
plt.title("Visualization of the clustering with TSNE", fontsize="25")



Graphically the clusters are distinctive enough. Let's take a closer look at the clusters that contain few customers.

**`Cluster 0`**

In [ ]:
final_dataset[final_dataset['cluster']==0]

In [ ]:
final_dataset[final_dataset['cluster']==0].mean()

In [ ]:
temp_final_df = final_dataset.reset_index()

In [ ]:
cust0 = list(temp_final_df[temp_final_df['cluster']==0]['UserId'])

In [ ]:
cluster0 = data[data['UserId'].isin(cust0)]
cluster0[['NumberOfItemsPurchased', 'CostPerItem','sales', 'frequency', 'min_recency'
         , 'monetary_value']].mean()

In [ ]:
cluster0['ItemDescription'].value_counts()[:10]

In [ ]:
custom_aggregation = {}
custom_aggregation["Country"] = lambda x:x.iloc[0]
custom_aggregation["RFMScore"] = lambda x:x.iloc[0]

cluster0_grouped = cluster0.groupby("UserId").agg(custom_aggregation)

In [ ]:
cluster0_grouped['RFMScore'].value_counts()

In [ ]:
cluster0_grouped['Country'].value_counts()

In [ ]:
cluster0['Month'].value_counts()

In [ ]:
cluster0['Hour'].value_counts()

In [ ]:
cluster0['Weekday'].value_counts()

**`Cluster 1`**

In [ ]:
final_dataset[final_dataset['cluster']==1]

In [ ]:
final_dataset[final_dataset['cluster']==1].mean()

In [ ]:
temp_final_df = final_dataset.reset_index()

In [ ]:
cust1 = list(temp_final_df[temp_final_df['cluster']==1]['UserId'])

In [ ]:
cluster1 = data[data['UserId'].isin(cust1)]
cluster1[['NumberOfItemsPurchased', 'CostPerItem','sales', 'frequency', 'min_recency'
         , 'monetary_value']].mean()

In [ ]:
cluster1['ItemDescription'].value_counts()[:10]

In [ ]:
custom_aggregation = {}
custom_aggregation["Country"] = lambda x:x.iloc[0]
custom_aggregation["RFMScore"] = lambda x:x.iloc[0]

cluster1_grouped = cluster1.groupby("UserId").agg(custom_aggregation)

In [ ]:
cluster1_grouped['RFMScore'].value_counts()

In [ ]:
cluster1_grouped['Country'].value_counts()

In [ ]:
cluster1['Month'].value_counts()

In [ ]:
cluster1['Hour'].value_counts()

In [ ]:
cluster1['Weekday'].value_counts()

**`Cluster 2`**

In [ ]:
final_dataset[final_dataset['cluster']==2]

In [ ]:
final_dataset[final_dataset['cluster']==2].mean()

In [ ]:
temp_final_df = final_dataset.reset_index()

In [ ]:
cust2 = list(temp_final_df[temp_final_df['cluster']==2]['UserId'])

In [ ]:
cluster2 = data[data['UserId'].isin(cust0)]
cluster2[['NumberOfItemsPurchased', 'CostPerItem','sales', 'frequency', 'min_recency'
         , 'monetary_value']].mean()

In [ ]:
cluster2['ItemDescription'].value_counts()[:10]

In [ ]:
custom_aggregation = {}
custom_aggregation["Country"] = lambda x:x.iloc[0]
custom_aggregation["RFMScore"] = lambda x:x.iloc[0]

cluster2_grouped = cluster2.groupby("UserId").agg(custom_aggregation)

In [ ]:
cluster2_grouped['RFMScore'].value_counts()

In [ ]:
cluster2_grouped['Country'].value_counts()

In [ ]:
cluster2['Month'].value_counts()

In [ ]:
cluster2['Hour'].value_counts()

In [ ]:
cluster2['Weekday'].value_counts()

**`Cluster 3`**

In [ ]:
final_dataset[final_dataset['cluster']==3]

In [ ]:
final_dataset[final_dataset['cluster']==3].mean()

In [ ]:
temp_final_df = final_dataset.reset_index()

In [ ]:
cust3 = list(temp_final_df[temp_final_df['cluster']==3]['UserId'])

In [ ]:
cluster3 = data[data['UserId'].isin(cust3)]
cluster3[['NumberOfItemsPurchased', 'CostPerItem','sales', 'frequency', 'min_recency'
         , 'monetary_value']].mean()

In [ ]:
cluster3['ItemDescription'].value_counts()[:10]

In [ ]:
custom_aggregation = {}
custom_aggregation["Country"] = lambda x:x.iloc[0]
custom_aggregation["RFMScore"] = lambda x:x.iloc[0]

cluster3_grouped = cluster3.groupby("UserId").agg(custom_aggregation)

In [ ]:
cluster3_grouped['RFMScore'].value_counts()

In [ ]:
cluster3_grouped['Country'].value_counts()

In [ ]:
cluster3['Month'].value_counts()

In [ ]:
cluster3['Hour'].value_counts()

In [ ]:
cluster3['Weekday'].value_counts()

**`Cluster 4`**

In [ ]:
final_dataset[final_dataset['cluster']==4]

In [ ]:
final_dataset[final_dataset['cluster']==4].mean()

In [ ]:
temp_final_df = final_dataset.reset_index()

In [ ]:
cust4 = list(temp_final_df[temp_final_df['cluster']==4]['UserId'])

In [ ]:
cluster4 = data[data['UserId'].isin(cust4)]
cluster4[['NumberOfItemsPurchased', 'CostPerItem','sales', 'frequency', 'min_recency'
         , 'monetary_value']].mean()

In [ ]:
cluster4['ItemDescription'].value_counts()[:10]

In [ ]:
custom_aggregation = {}
custom_aggregation["Country"] = lambda x:x.iloc[0]
custom_aggregation["RFMScore"] = lambda x:x.iloc[0]

cluster4_grouped = cluster4.groupby("UserId").agg(custom_aggregation)

In [ ]:
cluster4_grouped['RFMScore'].value_counts()

In [ ]:
cluster4_grouped['Country'].value_counts()

In [ ]:
cluster4['Month'].value_counts()

In [ ]:
cluster4['Hour'].value_counts()

In [ ]:
cluster4['Weekday'].value_counts()

**`Cluster 5`**

In [ ]:
final_dataset[final_dataset['cluster']==5]

In [ ]:
final_dataset[final_dataset['cluster']==5].mean()

In [ ]:
temp_final_df = final_dataset.reset_index()

In [ ]:
cust5 = list(temp_final_df[temp_final_df['cluster']==5]['UserId'])

In [ ]:
cluster5 = data[data['UserId'].isin(cust5)]
cluster5[['NumberOfItemsPurchased', 'CostPerItem','sales', 'frequency', 'min_recency'
         , 'monetary_value']].mean()

In [ ]:
cluster5['ItemDescription'].value_counts()[:10]

In [ ]:
custom_aggregation = {}
custom_aggregation["Country"] = lambda x:x.iloc[0]
custom_aggregation["RFMScore"] = lambda x:x.iloc[0]

cluster5_grouped = cluster5.groupby("UserId").agg(custom_aggregation)

In [ ]:
cluster5_grouped['RFMScore'].value_counts()

In [ ]:
cluster5_grouped['Country'].value_counts()

In [ ]:
cluster5['Month'].value_counts()

In [ ]:
cluster5['Hour'].value_counts()

In [ ]:
cluster5['Weekday'].value_counts()

**`Cluster 6`**

In [ ]:
final_dataset[final_dataset['cluster']==6]

In [ ]:
final_dataset[final_dataset['cluster']==6].mean()

In [ ]:
temp_final_df = final_dataset.reset_index()

In [ ]:
cust6 = list(temp_final_df[temp_final_df['cluster']==6]['UserId'])

In [ ]:
cluster6 = data[data['UserId'].isin(cust6)]
cluster6[['NumberOfItemsPurchased', 'CostPerItem','sales', 'frequency', 'min_recency'
         , 'monetary_value']].mean()

In [ ]:
cluster6['ItemDescription'].value_counts()[:10]

In [ ]:
custom_aggregation = {}
custom_aggregation["Country"] = lambda x:x.iloc[0]
custom_aggregation["RFMScore"] = lambda x:x.iloc[0]

cluster6_grouped = cluster6.groupby("UserId").agg(custom_aggregation)

In [ ]:
cluster6_grouped['RFMScore'].value_counts()

In [ ]:
cluster6_grouped['Country'].value_counts()

In [ ]:
cluster6['Month'].value_counts()

In [ ]:
cluster6['Hour'].value_counts()

In [ ]:
cluster6['Weekday'].value_counts()

**`Cluster 7`**

In [ ]:
final_dataset[final_dataset['cluster']==7]

In [ ]:
final_dataset[final_dataset['cluster']==7].mean()

In [ ]:
temp_final_df = final_dataset.reset_index()

In [ ]:
cust7 = list(temp_final_df[temp_final_df['cluster']==7]['UserId'])

In [ ]:
cluster7 = data[data['UserId'].isin(cust7)]
cluster7[['NumberOfItemsPurchased', 'CostPerItem','sales', 'frequency', 'min_recency'
         , 'monetary_value']].mean()

In [ ]:
cluster7['ItemDescription'].value_counts()[:10]

In [ ]:
custom_aggregation = {}
custom_aggregation["Country"] = lambda x:x.iloc[0]
custom_aggregation["RFMScore"] = lambda x:x.iloc[0]

cluster7_grouped = cluster7.groupby("UserId").agg(custom_aggregation)

In [ ]:
cluster7_grouped['RFMScore'].value_counts()

In [ ]:
cluster7_grouped['Country'].value_counts()

In [ ]:
cluster7['Month'].value_counts()

In [ ]:
cluster7['Hour'].value_counts()

In [ ]:
cluster7['Weekday'].value_counts()

In [ ]:
cluster0_={'cluster0':cluster0['UserId'].dropna().unique()
          }
cluster0_=pd.DataFrame(cluster0_,columns=['cluster0'])

cluster1_={'cluster1':cluster1['UserId'].dropna().unique()
          }
cluster1_=pd.DataFrame(cluster1_,columns=['cluster1'])


cluster2_={'cluster2':cluster2['UserId'].dropna().unique()
          }
cluster2_=pd.DataFrame(cluster2_,columns=['cluster2'])


cluster3_={'cluster3':cluster3['UserId'].dropna().unique()
          }
cluster3_=pd.DataFrame(cluster3_,columns=['cluster3'])



cluster4_={'cluster4':cluster4['UserId'].dropna().unique()
          }
cluster4_=pd.DataFrame(cluster4_,columns=['cluster4'])


cluster5_={'cluster5':cluster5['UserId'].dropna().unique()
          }
cluster5_=pd.DataFrame(cluster5_,columns=['cluster5'])



cluster6_={'cluster6':cluster6['UserId'].dropna().unique()
          }
cluster6_=pd.DataFrame(cluster6_,columns=['cluster6'])



cluster7_={'cluster7':cluster7['UserId'].dropna().unique()
          }
cluster7_=pd.DataFrame(cluster7_,columns=['cluster7'])


# cluster0_.to_csv('cluster0-Best_Customers.csv',index=False)
# cluster1_.to_csv('cluster1-Lost_Cheap_Customers.csv',index=False)
# cluster2_.to_csv('cluster2-Bestest_Customer.csv',index=False)
# cluster3_.to_csv('cluster3-Bestest_Customers.csv',index=False)
# cluster4_.to_csv('cluster4-Lost_Customers.csv',index=False)
# cluster5_.to_csv('cluster5-Foreign_NonUK_Customers.csv',index=False)
# cluster6_.to_csv('cluster6_Big_Spenders.csv',index=False)
# cluster7_.to_csv('cluster7-Loyal_and_Big_Spenders.csv',index=False)


# print("All clusters are saved as CSV files")